In [1]:
import plotly.graph_objects as go
import numpy as np
from numpy import pi, radians
from datetime import date


from func.rotation_z import R_z
from func.transition_matrix import transition_matrix
from func.rotate_matrix import rotate_matrix
from func.init_points import init_points
from func.cross_point import cross_point

# Нахождение точки пересечения тяги и рычага тренажёра

## 1 Общая запись матричных преобразований


**С помощью ниже представленного матричного вычисления мы перемещаем И точку, И её локальную систему координат.**
$$

\Bigg(
  \begin{vmatrix}
  \ 1& \ 0& \ 0& \ Δx \\
  \ 0& \ 1& \ 0& \ Δy \\
  \ 0& \ 0& \ 1& \ Δz \\
  \ 0& \ 0& \ 0& \ 1 \\
\end{vmatrix}
\times
\begin{vmatrix}
  \ cos(ψ)& \ -sin(ψ)& \ 0& \ 0 \\
  \ sin(ψ)& \ cos(ψ)& \ 0& \ 0 \\
  \ 0& \ 0& \ 1& \ 0 \\
  \ 0& \ 0& \ 0& \ 1 \\
\end{vmatrix}\Bigg)
\times
\begin{vmatrix}
  \ x_0\\
  \ y_0\\
  \ z_0\\
  \ 0\\
\end{vmatrix}
= (A \times B) \times С = D,
$$

где A - матрица перемещения, B - матрица поворота, C - начальные координаты точки.

## 2 Начальные координаты точки 
$$
\begin{vmatrix}
  \ x_0\\
  \ y_0\\
  \ z_0\\
  \ 0\\
\end{vmatrix} = C
$$

Найдём точку пересечения 1-ой группы тяга-рычаг. Пусть крепление рычага (нижняя платформа) изначально находится в центре глобальной СК, тогда $(x_0, y_0, z_0)$ = (0, 0, 0)

In [2]:
first_lower_point = init_points(0, 0, 0)
# Важно уточнить, что тут могут быть абсолютно любые координаты

## 3 Создадим матрицу переноса точки крепления рычага (нижняя платформа)

$$
\begin{vmatrix}
  \ 1& \ 0& \ 0& \ Δx \\
  \ 0& \ 1& \ 0& \ Δy \\
  \ 0& \ 0& \ 1& \ Δz \\
  \ 0& \ 0& \ 0& \ 1 \\
\end{vmatrix}
\times
\begin{vmatrix}
  \ cos(ψ)& \ -sin(ψ)& \ 0& \ 0 \\
  \ sin(ψ)& \ cos(ψ)& \ 0& \ 0 \\
  \ 0& \ 0& \ 1& \ 0 \\
  \ 0& \ 0& \ 0& \ 1 \\
\end{vmatrix}= A \times B
$$

Угол ψ (в нашем случае) - это угол OZ на который мы хотим повернуть локальную СК ДО перемещения. Если Вы хотите сначала переместиться, а затем повернуть СК, то измените порядок умножения матриц. (Δx, Δy, Δz) - это линейные координаты, на которые необходимо сместить точку. 

В plotly OZ направленна вверх. Мы будем считать, что в нашей локальной СК OZ - это ось поворота рычага, а OY вертикальна. Для этого сначала мы перемещаем точку, а затем поворачиваем её СК.


In [3]:
from_base_CS_to_lower_CS = transition_matrix(35, 8.66 , 0).dot(rotate_matrix(0, 0, radians(60))) 

Если у Вас несколько поворотов, просто перемножьте матрицы этих поворотов, т.е.:
$$
A \times (B_1 \times B_2 \times ...)
$$

## 4 Нахождение точки нижней платформы через полученную матрицу переноса

Умножив матрицу перемещения на начальные координаты найдём координаты после переноса. Мы выбрали за начало (0, 0, 0), но могли выбрать любые другие координаты.

In [4]:
new_position_lower_CS = from_base_CS_to_lower_CS.dot(first_lower_point)  

print(new_position_lower_CS)    # Нашли координаты нижней 4-ой точки

[[35.  ]
 [ 8.66]
 [ 0.  ]
 [ 1.  ]]


## 5 Найдём координаты верхней точки крепления тяги

Мы выбрали 4-ую точку нижней платформы, она должна соединяться с 4-ой точкой верхней платформы.

Найдём матрицу перемещения верхней точки. Мы считаем, что верхняя точка получается за счёт смещения 3-ей точки нижней платформы Δz = 50 и поворота по OZ = 60 град. 3-я точка платформы симметрична 4-ой относительно ZX => имеет полученные координаты со знаком "-" у координаты Y.

In [5]:
new_position_3_lower_CS = new_position_4_lower_CS
new_position_3_lower_CS[1][0] *= -1

from_base_CS_to_4_upper_CS = transition_matrix(0, 0, 50).dot(rotate_matrix(0, 0, radians(60)))

Перемножаем полученную матрицу на координаты точки нижней платформы

In [6]:
new_position_4_upper_CS = from_base_CS_to_4_upper_CS.dot(new_position_4_lower_CS)  

print(new_position_4_upper_CS)

[[24.99978   ]
 [25.98088913]
 [50.        ]
 [ 1.        ]]


## Определим координаты верхней точки, относительно нижней

In [7]:
position_upper_CS_from_lower_CS = np.linalg.inv(from_base_CS_to_4_lower_CS).dot(new_position_4_upper_CS)

print(position_upper_CS_from_lower_CS)

[[10.00022   ]
 [17.32088913]
 [50.        ]
 [ 1.        ]]


### Найдём точку пересечения тяги и окружности рычага (подробно это было расписано в 2. Sphere&circle.ipynb):

In [8]:
R = 20    # Радиус окружности
r = 40   # Радиус сферы

In [10]:
cross_matrix = cross_point(position_upper_CS_from_lower_CS, R, r)

cross_sphere_and_circle = init_points(cross_matrix[1][0], cross_matrix[1][1], cross_matrix[1][2])

print(cross_sphere_and_circle)

[[ 0.        ]
 [-6.00015332]
 [19.07873581]
 [ 1.        ]]


Получив координаты пересечения тяги и рычага в локальной системе координат переведём их в глобальную:

In [11]:
# Эти локальные координаты являются частью объекта, который мы и переносим
position_leg_4 = from_base_CS_to_4_lower_CS.dot(cross_sphere_and_circle)
print(position_leg_4)

[[40.1962852 ]
 [ 5.65992334]
 [19.07873581]
 [ 1.        ]]


## Отрисовка

In [12]:
lower_plate = np.array([[-25, -10, 35, 35, -10, -25, -25], # Координаты вершин нижней плиты 
                [-25.98, -34.64, -8.66, 8.66, 34.64, 25.98, -25.98],
                [0, 0, 0, 0, 0, 0, 0]]) 

# Центр верхней плиты
xyz_0 = np.array([[0], [0], [50]])      

# Положение координат углов верхней плиты относительно её центра
upper_plate = np.array([[float(xyz_0[0])+-25, float(xyz_0[0])+-10, float(xyz_0[0])+35, float(xyz_0[0])+35, float(xyz_0[0])+-10, float(xyz_0[0])+-25, float(xyz_0[0])+-25], # Координаты вершин нижней плиты
                [float(xyz_0[1])+-25.98, float(xyz_0[1])+-34.64, float(xyz_0[1])+-8.66, float(xyz_0[1])+8.66, float(xyz_0[1])+34.64, float(xyz_0[1])+25.98, float(xyz_0[1])+-25.98],
                [float(xyz_0[2])+0, float(xyz_0[2])+0, float(xyz_0[2])+0, float(xyz_0[2])+0, float(xyz_0[2])+0, float(xyz_0[2])+0, float(xyz_0[2])+0]])
                  
upper_plate = R_z(60, upper_plate) # Поворот верхней плиты на 60 град  

In [13]:
fig = go.Figure()
      # Нижняя плита
fig.add_trace(go.Scatter3d(x=[0], y=[0], z=[0],  
                showlegend=False, surfaceaxis=-1, opacity=1, legendgroup="group1",marker=dict(size=5, color='red', colorscale='Viridis',)))
fig.add_trace(go.Scatter3d(x=lower_plate[0], y=lower_plate[1], z=lower_plate[2],    
                showlegend=True, name = 'Нижняя плита', surfaceaxis=2,  opacity=0.3, legendgroup="group1",marker=dict(size=1, color='red', colorscale='Viridis',)))
fig.add_trace(go.Scatter3d(x=lower_plate[0], y=lower_plate[1], z=lower_plate[2],
                showlegend=False, surfaceaxis=-1, opacity=1, legendgroup="group1",marker=dict(size=1, color='black', colorscale='Viridis',)))     
#_____________

# Верхняя плита
fig.add_trace(go.Scatter3d(x=xyz_0[0], y=xyz_0[1], z=xyz_0[2], 
                showlegend=False, surfaceaxis=-1, opacity=1, legendgroup="group2",marker=dict(size=5, color='blue', colorscale='Viridis',))) 
fig.add_trace(go.Scatter3d(x=upper_plate[0], y=upper_plate[1], z=upper_plate[2],    
                showlegend=True, name = 'Верхняя плита', surfaceaxis=2,  opacity=0.3, legendgroup="group2",marker=dict(size=1, color='royalblue', colorscale='Viridis',)))
fig.add_trace(go.Scatter3d(x=upper_plate[0], y=upper_plate[1], z=upper_plate[2],
                showlegend=False, surfaceaxis=-1, opacity=1, legendgroup="group2",marker=dict(size=1, color='black', colorscale='Viridis',)))      
#_____________

# Нога 4              
fig.add_trace(go.Scatter3d(x=[lower_plate[0][3], position_leg_4[0][0]], y=[lower_plate[1][3], position_leg_4[1][0]], z=[lower_plate[2][3], position_leg_4[2][0]],  
                showlegend=True, name = f'Рычаг-Тяга 4 ({round(cross_matrix[1][3]*180/pi, 2)}, град)', opacity=1, legendgroup="group6",marker=dict(size=5, color='orange', colorscale='Viridis',)))
fig.add_trace(go.Scatter3d(x=[position_leg_4[0][0], upper_plate[0][2]], y=[position_leg_4[1][0], upper_plate[1][2]], z=[position_leg_4[2][0], upper_plate[2][2]],  
                showlegend=False, opacity=1, legendgroup="group6",marker=dict(size=5, color='orange', colorscale='Viridis')))  
  
#_____________ 

fig.update_layout(title={'text': f"{date.today()}",
                  'y':0.97,'x':0.5,'xanchor': 'center','yanchor': 'top'})

                                             #Оформление
fig.update_layout(legend=dict(yanchor="top", y=0.99,
                  xanchor="left", x=0.7))

tickf = 12
fig.update_layout(autosize=True, scene = {'camera_eye': {"x": -2, "y": 2, "z":1.65},'camera_center' : {"x": 0, "y": 0, "z":0},}, 
                  width=950, height=500, margin=dict(l=10, r=0, b=10, t=50))

fig.update_layout(scene=dict(xaxis=dict( title="X",backgroundcolor="rgb(200, 200, 230)",gridcolor="white",
                         showbackground=True,zerolinecolor="white", tickfont=dict(size=tickf)),
                  yaxis=dict(title="Y", backgroundcolor="rgb(230, 200,230)",gridcolor="white",
                        showbackground=True,tickfont=dict(size=tickf),zerolinecolor="white"),
                  zaxis=dict(title= "Z", backgroundcolor="rgb(200, 200,200)",gridcolor="white",
                         showbackground=True,tickfont=dict(size=tickf),zerolinecolor="white",)))

fig.update_layout(scene=dict(xaxis_showspikes=False, yaxis_showspikes=False),)
fig.update_scenes(camera_projection_type="orthographic")

fig.show()